In [1]:
require 'nn'
require 'hdf5'

### LOADING THE DATA AND CONVERTING IT TO LOGREG FW:

In [2]:
myFile = hdf5.open('PTB.hdf5','r')

In [3]:
data = myFile:all()
myFile:close()

In [ ]:
data

In [4]:
train_input_word_windows = data['train_input_word_windows']
train_output = data['train_output']

In [5]:
train = train_input_word_windows:clone()

In [6]:
for i = 1, train:size(1) do
    for j = 1, 5 do
        train[i][j] = (j-1)*100002 + train[i][j]
    end
end

In [56]:
train_input_cap_windows = data['train_input_cap_windows']
train_cap = train_input_cap_windows:clone()
for i = 1, train:size(1) do
    for j = 1, 5 do
        train_cap[i][j] = (j-1)*4 + train_cap[i][j]
    end
end

### TRAINING:

In [50]:
linreg = nn.Sequential()

In [45]:
par = nn.ParallelTable()

In [46]:
par:add(nn.LookupTable(5*data['nwords'][1],data['nclasses'][1])) -- first child
par:add(nn.LookupTable(5*4,data['nclasses'][1])) 

In [47]:
temp = nn.Sequential()
temp:add(par)
temp:add(nn.CAddTable())
temp:add(nn.Sum(2))

In [36]:
-- test2 = nn.Sequential()
-- test2:add(nn.LookupTable(5*data['nwords'][1],data['nclasses'][1]))
-- test2:add(nn.Sum(2))

-- test3 = nn.Sequential()
-- test3:add(nn.LookupTable(20,data['nclasses'][1]))
-- test3:add(nn.Sum(2))

-- print(test2:forward(torch.LongTensor({{1,3,6},{2,4,6}})) + test2:forward(torch.LongTensor({{1,3,6},{2,4,6}})))

(1,.,.) = 
 Columns 1 to 9
 -7.2710 -1.7081 -3.1657 -1.8297 -1.1900 -4.5574  0.6756 -2.2598 -2.3324

Columns 10 to 18
 -1.3096  6.9826 -7.0159 -3.9894 -7.9360  5.0311  0.0208 -3.1380 -2.1154

Columns 19 to 27
  4.2404  2.9047 -1.5363 -3.3677 -2.7965  1.8091  0.6596  1.9847  7.1568

Columns 28 to 36
  3.3107  0.5776 -1.0683  1.0073 -0.3328 -0.2753 -0.3806  1.6605  1.0147

Columns 37 to 45
 -1.5222  0.9538  3.0052  2.1032 -1.5745  3.9696  2.0968  5.6390 -1.2959

(2,.,.) = 
 Columns 1 to 9
 -3.9518 -1.1080 -3.3476  4.8042  1.0943 -5.9027  1.4511 -6.3428  3.8737

Columns 10 to 18
 -0.0832  2.6380  0.2768 -6.2396 -0.1083  7.8906 -6.7006  2.5848  0.7967

Columns 19 to 27
  0.3524 -1.9435  4.4511  5.9728  1.9779  2.1402 -0.3527 -1.6950  1.2584

Columns 28 to 36
 -1.0651  0.8247 -3.5832  0.2430 -1.6710  6.3113  1.1022 -0.4977 -0.1803

Columns 37 to 45
  3.1177  3.7266  1.4428 -3.7968  0.5541  2.0230 -0.1904 -2.2897 -3.4696
[torch.DoubleTensor of size 2x1x45]



In [ ]:
-- linreg:add(nn.LookupTable(5*data['nwords'][1],data['nclasses'][1]))
-- linreg:add(nn.Sum(2))
-- linreg:add(nn.Add(data['nclasses'][1]))
-- linreg:add(nn.LogSoftMax())

In [51]:
linreg:add(temp)
linreg:add(nn.Add(data['nclasses'][1]))
linreg:add(nn.LogSoftMax)

In [54]:
train:narrow(1,5, 10)

      4  100228  200089  300176  400012
    226  100087  200174  300010  400054
     85  100172  200008  300052  401440
    170  100006  200050  301438  400011
      4  100048  201436  300009  400545
     46  101434  200007  300543  400030
   1432  100005  200541  300028  400018
      3  100539  200026  300016  400010
    537  100024  200014  300008  400380
     22  100012  200006  300378  402354
[torch.LongTensor of size 10x5]



In [58]:
train_cap:narrow(1,5, 10)

  1   6   9  13  17
  2   5   9  13  17
  1   5   9  13  17
  1   5   9  13  17
  1   5   9  13  17
  1   5   9  13  17
  1   5   9  13  17
  1   5   9  13  18
  1   5   9  14  17
  1   5  10  13  17
[torch.LongTensor of size 10x5]



In [57]:
linreg:forward({train:narrow(1,5, 10),train_cap:narrow(1,5, 10)})

...irgileaudi/torch/install/share/lua/5.1/nn/LogSoftMax.lua:6: attempt to index field 'output' (a nil value)
stack traceback:
	...irgileaudi/torch/install/share/lua/5.1/nn/LogSoftMax.lua:6: in function 'updateOutput'
	...irgileaudi/torch/install/share/lua/5.1/nn/Sequential.lua:44: in function 'f'
	[string "local f = function() return linreg:forward({t..."]:1: in main chunk
	[C]: in function 'xpcall'
	.../virgileaudi/torch/install/share/lua/5.1/itorch/main.lua:179: in function <.../virgileaudi/torch/install/share/lua/5.1/itorch/main.lua:143>
	.../virgileaudi/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...rgileaudi/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...rgileaudi/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...rgileaudi/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	.../virgileaudi/torch/install/share/lua/5.1/itorch/main.lua:350: in main chunk
	[C]: in function 'require'
	

In [52]:
criterion = nn.ClassNLLCriterion()

In [ ]:
train_output:size(1)

In [ ]:
eta = 0.01
input = torch.Tensor(100,5)
output = torch.Tensor(100)

for i = 1,5 do
   
    --28520
    for j = 1,9126 do
        linreg:zeroGradParameters()
        
        input = train:narrow(1,(j-1)*100+1, 100)
        preds = linreg:forward(input)
        
        output = train_output:narrow(1,(j-1)*100+1, 100)
        
        loss = criterion:forward(preds, output)
        
        if j % 500 == 0 then
            print(loss)
        end
        
        dLdpreds = criterion:backward(preds, output)
        
        linreg:backward(input, dLdpreds)
        
        linreg:updateParameters(eta)
        
    end
    
end


### VALIDATION:

In [ ]:
val = data['valid_input_word_windows']:clone()
for i = 1, val:size(1) do
    for j = 1, 5 do
        val[i][j] = (j-1)*100002 + val[i][j]
    end
end

In [ ]:
pred_val = linreg:forward(val)

In [ ]:
m,a = pred_val:max(2)

In [ ]:
acc = 0
for i = 1, data['valid_output']:size(1) do
    if a[i] == data['valid_output'][i] then
        acc = acc + 1
    end
end
print(acc)

In [ ]:
minibatch =  torch.range((j-1)*32+1, j*32):type('torch.LongTensor')

In [ ]:
preds = linreg:forward(train:index(1,minibatch))

In [ ]:
loss = mse:forward(preds, train_output:index(1,minibatch))
        

In [ ]:
dLdpreds = mse:backward(preds, train_output:index(1,minibatch))

In [ ]:
linreg:backward(X, dLdpreds)

In [ ]:
x = torch.LongTensor({1,2,5})